In [1]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.preprocessing import RobustScaler
import pickle

In [19]:
df.head()

,TOP_TYPE,B_ACCELERATED,TYPE_OF_CONTRACT,TAL_LOCATION_NUTS,CPV,DT_AWARD,NUMBER_OFFERS,AWARD_VALUE_EURO_FIN_1,AWARD_EST_VALUE_EURO,date copy
0,OPE,N,W,BG314,31,2019-12-02,3.0,2657688,2671496.57,2019-12-02
1,OPE,N,W,BG422,45,2019-12-09,3.0,3058417,3067798.34,2019-12-09
2,OPE,N,U,EL434,38,2019-09-02,1.0,237356,242134.07,2019-09-02
3,OPE,N,U,PL623,48,2019-12-23,3.0,442493,389196.49,2019-12-23
4,OPE,N,U,PL,33,2019-12-16,1.0,70968,67499.67,2019-12-16


In [14]:
def encode(df):
    #copy date column for sorting
    df["date copy"] = df["DT_AWARD"].copy(deep=True)
    
    #ENCODE COLUMNS
    base_n_encoder_cols = ["CPV", "ISO_COUNTRY_CODE", "DT_AWARD", "TAL_LOCATION_NUTS"]
    one_hot_cols = ["TYPE_OF_CONTRACT", "TOP_TYPE", "B_ACCELERATED"]
    encoder = ce.BaseNEncoder(cols=base_n_encoder_cols, return_df=True, base=2)
    df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True, dtype=int)
    df = encoder.fit_transform(df)

    return df

In [15]:
def scale(df):
    numeric_cols = ["CRIT_PRICE_WEIGHT", "NUMBER_OFFERS", "AWARD_EST_VALUE_EURO"]
    scaler = RobustScaler()

    values = df[numeric_cols].values
    scaled_values = scaler.fit_transform(values)
    df_numeric_scaled = pd.DataFrame(data=scaled_values)
    df.loc[:,numeric_cols] = df_numeric_scaled

    return df

In [16]:
def encode_scale_split(df, train, validate):

    #ENCODE
    df = encode(df)

    target_col = "AWARD_VALUE_EURO_FIN_1"
    df = df.sort_values(by = ["date copy"], axis = 0, ascending = True)
    train_indice = int(train * len(df))
    validate_indice = train_indice + int(validate * len(df))

    train_set = df.iloc[:train_indice]
    val_set = df.iloc[train_indice:validate_indice]
    test_set = df.iloc[validate_indice:]

    #PERFORM SCALING
    #train_set = scale(train_set)
    #val_set = scale(val_set)
    #test_set = scale(test_set)

    #SPLIT THE DATASET
    X_train = train_set.drop(columns = [target_col]).values
    y_train = train_set[target_col].values

    X_val = val_set.drop(columns = [target_col]).values
    y_val = val_set[target_col].values

    X_test = test_set.drop(columns = [target_col]).values
    y_test = test_set[target_col].values

    return X_train, y_train, X_val, y_val, X_test, y_test

In [17]:
df = pd.read_pickle("../3.new_data/4A.df_structured")

In [18]:
X_train, y_train, X_val, y_val, X_test, y_test = encode_scale_split(df, 0.6, 0.2)

ValueError: X does not contain the columns listed in cols